In [139]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [140]:
df_clean = pd.read_pickle('df_clean_final.pkl')

df_clean

,age,sex,bmi,children,smoker,charges,is_southwest,is_southeast,is_northwest,is_northeast,age_cat,18_35,36_50,51_70,71_200,bmi_cat,underweight,normal,overweight,obesity
0,19,0,27.900,0,1,16884.92400,1,0,0,0,18_35,1,0,0,0,overweight,0,0,1,0
1,18,1,33.770,1,0,1725.55230,0,1,0,0,18_35,1,0,0,0,obesity,0,0,0,1
2,28,1,33.000,3,0,4449.46200,0,1,0,0,18_35,1,0,0,0,obesity,0,0,0,1
3,33,1,22.705,0,0,21984.47061,0,0,1,0,18_35,1,0,0,0,normal,0,1,0,0
4,32,1,28.880,0,0,3866.85520,0,0,1,0,18_35,1,0,0,0,overweight,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333,50,1,30.970,3,0,10600.54830,0,0,1,0,36_50,0,1,0,0,obesity,0,0,0,1
1334,18,0,31.920,0,0,2205.98080,0,0,0,1,18_35,1,0,0,0,obesity,0,0,0,1
1335,18,0,36.850,0,0,1629.83350,0,1,0,0,18_35,1,0,0,0,obesity,0,0,0,1
1336,21,0,25.800,0,0,2007.94500,1,0,0,0,18_35,1,0,0,0,overweight,0,0,1,0


In [154]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Lasso,ElasticNet,Ridge
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer

# import du csv
df = pd.read_csv('data/data.csv')

# selection des features et de la cible
X = df.drop('charges', axis=1)
y = df.charges

# Création du train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.85)


# Sélection des variables numériques
num_col = list(X.select_dtypes(include=[float,int]).columns)

# Sélection des variables catégorielles
cat_col = list(X.select_dtypes(include=[object]).columns)

# pipeline pour les valeurs numériques
# cela permet de créer une succession d'étapes (ici il n'y en a qu'une)

my_num_pipe = make_pipeline(StandardScaler())

my_cat_pipe = make_pipeline(OneHotEncoder())

# ColumnTransformer permet de faire des opérations sur des sélections de colonnes
preprocessing = ColumnTransformer([
    ("one_hot", my_cat_pipe,cat_col),
    ("scaling", my_num_pipe, num_col)
])

# # Ajuster le preprocessing sur les données d'entraînement
# preprocessing.fit(X)

# # Transformer les données d'entraînement
# X_transformed = preprocessing.transform(X)

# # Obtenir les noms de colonnes après la transformation
# transformed_columns = preprocessing.get_feature_names_out()

# # Créer un DataFrame avec les données transformées et les noms de colonnes
# df_X_train_transformed = pd.DataFrame(X_transformed, columns=transformed_columns)
# print(df_X_train_transformed.head())
# print(X.shape,X_train.shape)


# j'ajoute la régression linéaire au pipeline
my_pipe_lr = make_pipeline(preprocessing, LinearRegression())
my_pipe_lr.fit(X_train,y_train)
score_LR = my_pipe_lr.score(X_test,y_test)
print("Score LR :",score_LR)

# Je ré-utilise le pipeline mais avec lasso
my_pipe_lasso = make_pipeline(preprocessing, Lasso())
my_pipe_lasso.fit(X_train,y_train)
score_lasso = my_pipe_lasso.score(X_test,y_test)
print("score lasso :",score_lasso)

# Je ré-utilise le pipeline mais avec Ridge
my_pipe_Ridge = make_pipeline(preprocessing,Ridge())
my_pipe_Ridge.fit(X_train,y_train)
score_Ridge = my_pipe_Ridge.score(X_test,y_test)
print("Score Ridge :",score_Ridge)

# Je ré-utilise le pipeline mais avec Elasticnet

my_pipe_Elasticnet = make_pipeline(preprocessing,ElasticNet())
my_pipe_Elasticnet.fit(X_train,y_train)
score_Elasticnet = my_pipe_Elasticnet.score(X_test,y_test)
print("Score Elasticnet :",score_Elasticnet)


Score LR : 0.7790968407084968
score lasso : 0.7793028448083313
Score Ridge : 0.7793204052043158
Score Elasticnet : 0.5312226296480245


In [142]:
#Split des données
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42, stratify=X['smoker'])


# Regression lineaire 

## Regression Linéaire sans test ni train

In [143]:
y=df_clean[['charges']]
X = df_clean[['bmi','age','smoker','sex','children']]

# regression lineaire, calcule R2, scatterplot
model = LinearRegression()
model.fit(X,y)
model.score(X,y)
print("R2 (coefficient de détermination): ",model.score(X,y))
model.predict(X)
# plt.scatter(X,y)
# plt.plot(X,model.predict(X),c='r')

#calcul RSME

rsme= np.sqrt(mean_squared_error(y,model.predict(X)))
print("RSME: ",rsme)

R2 (coefficient de détermination):  0.7495540902592981
RSME:  6058.310641972325


En calculant ces deux métriques nous pouvons dire que notre algorithme se trompe en moyenne de 6058.31€ et que 74.96% de la variation de la charge est expliqué par le BMI.

In [144]:
from sklearn.model_selection import train_test_split

y=df_clean[['charges']]
X = df_clean[['bmi','age','smoker','sex','children']]

# regression lineaire, calcule R2, scatterplot

model = LinearRegression()
model.fit(X_train,y_train)
model.score(X_train,y_train)
print("R2 (coefficient de détermination): ",model.score(X_test,y_test))

# y_pred=model.predict(X_test)

# plt.scatter(X_test['bmi'],y_test)

# plt.scatter(X_test['bmi'],y_pred,color='red')
# # plt.plot(X,model.predict(X),c='r')

# #calcul RSME

# rmse= np.sqrt(mean_squared_error(y,model.predict(X)))
# print("RSME: ",rmse)

R2 (coefficient de détermination):  0.8282046626546052


### Regression ElasticNet

In [145]:
# Regression ElasticNet ChatGPT

from sklearn.model_selection import GridSearchCV

# Définir les valeurs à tester pour alpha et l1_ratio
param_grid = {'alpha': np.arange(0.1, 1.1, 0.1),
              'l1_ratio': np.arange(0.1, 1.1, 0.1)}

# Créer le modèle ElasticNet
model = ElasticNet()

# Créer l'objet GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5)  # cv spécifie le nombre de plis pour la validation croisée

# Effectuer la recherche sur les hyperparamètres
grid_search.fit(X_train, y_train)

# Afficher les meilleures valeurs trouvées
print("Meilleurs paramètres :", grid_search.best_params_)

# Utiliser le modèle avec les meilleures valeurs
best_model = grid_search.best_estimator_

# Évaluer le modèle
score = best_model.score(X_test, y_test)
print("Score :", score)


Meilleurs paramètres : {'alpha': 1.0, 'l1_ratio': 1.0}
Score : 0.8282124579134139


In [151]:
from sklearn.linear_model import ElasticNet


model= ElasticNet (alpha=1, l1_ratio =1)
model.fit(X_train, y_train)
score = model.score(X_test, y_test)
print(score)
y_pred = model.predict(X_test)

#calcul RSME

rmse= np.sqrt(mean_squared_error(y,model.predict(X)))
print("RSME: ",rmse)

0.7329541655792376
RSME:  6061.77851082753


### Regression Lasso

In [147]:
#Lasso Chatgpt

# Regression Lasso

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
import numpy as np

# Définir les valeurs à tester pour alpha
param_grid = {'alpha': np.arange(0.1, 1.1, 0.1)}

# Créer le modèle Lasso
model = Lasso()

# Créer l'objet GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5)

# Effectuer la recherche sur les hyperparamètres
grid_search.fit(X_train, y_train)

# Afficher les meilleures valeurs trouvées
print("Meilleurs paramètres :", grid_search.best_params_)

# Utiliser le modèle avec les meilleures valeurs
best_model = grid_search.best_estimator_

# Évaluer le modèle
score = best_model.score(X_test, y_test)
print("Score :", score)


Meilleurs paramètres : {'alpha': 1.0}
Score : 0.8282124579134139


In [148]:

from sklearn.linear_model import Lasso

lasso = Lasso(alpha=1)
lasso.fit(X_train, y_train)
score = lasso.score(X_test,y_test)

score

0.8282124579134139

### Regression Ridge

In [149]:
# Regression Ridge

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
import numpy as np

# Définir les valeurs à tester pour alpha
param_grid = {'alpha': np.arange(0.1, 1.1, 0.1)}

# Créer le modèle Ridge
model = Ridge()

# Créer l'objet GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5)

# Effectuer la recherche sur les hyperparamètres
grid_search.fit(X_train, y_train)

# Afficher les meilleures valeurs trouvées
print("Meilleurs paramètres :", grid_search.best_params_)

# Utiliser le modèle avec les meilleures valeurs
best_model = grid_search.best_estimator_

# Évaluer le modèle
score = best_model.score(X_test, y_test)
print("Score :", score)


Meilleurs paramètres : {'alpha': 0.7000000000000001}
Score : 0.8281403985821878


In [150]:
from sklearn.linear_model import Ridge

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=0)
ridge = Ridge(alpha=1.0)
ridge.fit(X_train,y_train)
score= ridge.score(X_test,y_test)
score


0.7330458256282031